In [ ]:
!pip install numpy==1.24.0 scikit-learn==1.3.0 pandas==2.0.3 lightautoml==0.4.1

In [ ]:
# Импортируем необходимые библиотеки
from sklearn.datasets import fetch_openml
import pandas as pd
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Загружаем данные Titanic из sklearn
titanic = fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame

# Подготовка данных (аналогично Kaggle Titanic)
# Создаем целевую переменную 'Survived' (1 - выжил, 0 - нет)
df['Survived'] = (df['survived'] == '1').astype(int)
df.drop('survived', axis=1, inplace=True)

# Разделяем на train и test (в sklearn нет отдельного test, создадим искусственно)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Survived'])

# Выделяем целевую переменную 'Survived'
TARGET_NAME = 'Survived'

# Дальше ваш код остается практически без изменений
train, valid = train_test_split(train_data,
                                test_size=0.2,
                                random_state=42,
                                stratify=train_data[TARGET_NAME].values)

automl = TabularAutoML(task=Task('binary'),
                       timeout=300,
                       cpu_limit=4)

# Укажем роли переменных (добавим 'drop' для нерелевантных колонок)
roles = {'target': TARGET_NAME,
         'drop': ['boat', 'body', 'home.dest']}  # эти колонки не было в Kaggle версии

automl.fit_predict(train, roles=roles, verbose=1)

# Оценка качества
valid_true = valid[TARGET_NAME].values
valid_pred = (automl.predict(valid).data[:, 0] > 0.5).astype(int)
accuracy = accuracy_score(valid_true, valid_pred)
print(f'Accuracy на валидационной выборке: {accuracy:.4f}')

# Предсказание на тестовых данных
test_pred = automl.predict(test_data)
test_pred_int = (test_pred.data[:, 0] > 0.5).astype(int)

# Для сохранения результатов можно использовать:
results = pd.DataFrame({
    'PassengerId': test_data.index,  # в sklearn версии нет PassengerId, используем индекс
    'Survived': test_pred_int
})
results.to_csv('titanic_sklearn_submission.csv', index=False)
print('Файл с предсказаниями сохранен: titanic_sklearn_submission.csv')

In [ ]:
from sklearn.datasets import fetch_openml

# Загружаем датасет Titanic
titanic = fetch_openml("titanic", version=1)

# Получаем данные
data = titanic.data
target = titanic.target

# Выводим первые 5 строк данных
print(data.head())


In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Загружаем данные Titanic
titanic = fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame

# Подготовка данных
df['Survived'] = (df['survived'] == '1').astype(int)
df.drop('survived', axis=1, inplace=True)

# Разделяем данные
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Survived'])
train, valid = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data['Survived'])

# Укажем роли переменных
roles = {
    'target': 'Survived',
    'drop': ['boat', 'body', 'home.dest']
}

# Список моделей для тестирования
models_to_test = [
    'linear_l2',  # Линейная модель
    'lgb',        # LightGBM
    'lgb_tuned',  # Настроенный LightGBM
    'cb',         # CatBoost
    'cb_tuned',   # Настроенный CatBoost
    'rf',         # Random Forest
    # 'svm',        # Support Vector Machine
    # 'knn',        # k-Nearest Neighbors
    # 'linear',     # Линейная регрессия
    # 'elasticnet'  # ElasticNet

]

best_model = None
best_accuracy = 0
results = {}

for model_name in models_to_test:
    print(f"\n=== Обучение модели: {model_name} ===")

    # Настройки AutoML для одной модели без блендинга
    automl = TabularAutoML(
        task=Task('binary'),
        timeout=300,
        cpu_limit=4,
        general_params={'use_algos': [[model_name]]},  # Используем только одну модель
        reader_params={
            'n_folds': 3,  # Минимальное количество фолдов
            'random_state': 42
        }  # Отключаем кросс-валидацию
    )

    # Обучение
    oof_pred = automl.fit_predict(train, roles=roles, verbose=0)

    # Предсказание на валидации
    valid_pred = automl.predict(valid)
    valid_pred_int = (valid_pred.data[:, 0] > 0.5).astype(int)

    # Оценка качества
    accuracy = accuracy_score(valid['Survived'], valid_pred_int)
    results[model_name] = accuracy

    print(f"Accuracy для {model_name}: {accuracy:.4f}")

    # Сохраняем лучшую модель
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_name

# Выводим результаты
print("\n=== Результаты ===")
for model, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model}: {acc:.4f}")

print(f"\nЛучшая модель: {best_model} с accuracy {best_accuracy:.4f}")

# Предсказание на тестовых данных лучшей моделью
print(f"\n=== Делаем предсказания на тесте лучшей моделью ({best_model}) ===")
automl_best = TabularAutoML(
    task=Task('binary'),
    timeout=300,
    cpu_limit=4,
    general_params={'use_algos': [[best_model]]},
    reader_params={'n_folds': 3, 'random_state': 42}
)
automl_best.fit_predict(pd.concat([train, valid]), roles=roles, verbose=1)

test_pred = automl_best.predict(test_data)
test_pred_int = (test_pred.data[:, 0] > 0.5).astype(int)

# Сохраняем результаты
results_df = pd.DataFrame({
    'PassengerId': test_data.index,
    'Survived': test_pred_int
})
results_df.to_csv(f'titanic_best_model_{best_model}.csv', index=False)
print(f"Файл с предсказаниями сохранен: titanic_best_model_{best_model}.csv")

In [ ]:
print(f"\nЛучшая модель: {best_model} с accuracy {best_accuracy:.4f}")